In [8]:
import os
import pickle

import pandas as pd
import mlflow
import uuid

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [21]:
year = 2023
month = 3
taxi_type = "green"
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

In [3]:
RUN_ID = os.getenv('RUN_ID', '95182c88ec8040888af37b5f0259e733')
# RUN_ID = '95182c88ec8040888af37b5f0259e733'

In [4]:
#!mkdir output

In [5]:
#model

In [25]:
def generatre_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids
    
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)
   

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    n = len(df)
    df['ride_id'] = generatre_uuids(n)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [27]:
def load_model(run_id):
    logged_model = f's3://mlflows-artifacts-remote/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model
    
def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dict = prepare_dictionaries(df)
    model = load_model(run_id)
    y_pred = model.predict(dict)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = RUN_ID

    print(df_result.describe())
    
    df_result.to_parquet(output_file, index=False)

In [17]:
!mkdir output/green

In [28]:
apply_model(input_file=input_file,run_id=RUN_ID,output_file=output_file) 

           VendorID        lpep_pickup_datetime       lpep_dropoff_datetime  \
count  72044.000000                       72044                       72044   
mean       1.865443  2023-03-16 08:02:42.689842  2023-03-16 08:20:56.525748   
min        1.000000         2023-02-23 16:46:29         2023-02-23 16:55:25   
25%        2.000000  2023-03-08 13:55:49.500000  2023-03-08 14:15:59.750000   
50%        2.000000  2023-03-16 08:05:27.500000         2023-03-16 08:22:27   
75%        2.000000  2023-03-23 21:51:25.500000  2023-03-23 22:12:31.500000   
max        2.000000         2023-04-01 00:09:27         2023-04-01 17:32:21   
std        0.341252                         NaN                         NaN   

         RatecodeID  PULocationID  DOLocationID  passenger_count  \
count  67507.000000  72044.000000  72044.000000     67507.000000   
mean       1.145703     98.437413    137.613556         1.286844   
min        1.000000      1.000000      1.000000         0.000000   
25%        1.000

             lpep_pickup_datetime  actual_duration  predicted_duration  \
count                       69392     69392.000000        69392.000000   
mean   2023-03-16 08:14:33.108240        14.198222           13.173489   
min           2023-02-23 16:46:29         1.000000            1.542716   
25%    2023-03-08 14:07:42.750000         7.583333            7.814694   
50%           2023-03-16 08:09:56        11.733333           11.645506   
75%    2023-03-23 22:01:23.500000        18.000000           17.706375   
max           2023-04-01 00:09:27        60.000000           52.375735   
std                           NaN         9.573627            7.069541   

               diff  
count  69392.000000  
mean       1.024733  
min      -36.912111  
25%       -1.697512  
50%        0.166373  
75%        2.707153  
max       51.091654  
std        5.800414  


In [19]:
!ls output/green


green_tripdata_2021-02.parquet


In [20]:
!pwd

/home/ubuntu/mlopszoomcamp/04-deployment/batch


In [29]:
!mkdir output/yellow
   